In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import math

<h2> Read the file </h2>

In [2]:
# Read datafile corresponding to territorial emissions
df = pd.read_csv("../../../data/ghg-emissions/gcp/GCB2020v18_MtCO2_flat.csv")

# Set the dataframe structure
df.sort_values(["Year","Country"],inplace=True)
df.set_index(["Year","Country"],inplace=True)

# Convert the MtCO2 in MtC (to be in adequation with the values from other dataset of GCP)
df[["Total","Coal","Oil","Gas","Cement","Flaring","Other","Per Capita"]] = (1.0/3.664)* df[["Total","Coal","Oil","Gas","Cement","Flaring","Other","Per Capita"]]

df.tail(3)

ISO 3166-1 alpha-3     Total      Coal       Oil      Gas  \
Year Country                                                              
2019 Yemen                   YEM  2.798867  0.092000  2.137605  0.34078   
     Zambia                  ZMB  1.834195  0.512601  1.036221  0.00000   
     Zimbabwe                ZWE  2.831410  1.720523  0.952260  0.00000   

                 Cement  Flaring  Other  Per Capita  
Year Country                                         
2019 Yemen     0.228481      0.0    NaN    0.095977  
     Zambia    0.285373      0.0    NaN    0.102693  
     Zimbabwe  0.158627      0.0    NaN    0.193330

<h2>Mapping</h2>

In [20]:
# Option to select only a subset of countries/year (to check structure)
#list_country=["Afghanistan","Albania"]
#list_years=[2018,2019]

# Option to generate the full mapped dataset
list_years = df.index.get_level_values(0).unique()
list_country = df.index.get_level_values(1).unique()

# Definition of the sub-sector list
list_subsectors=["Coal","Oil","Gas","Cement","Flaring","Other"]


file = open("sample.json", "w")
# Loop over geo-component (in this case, countries)
for country in list_country:
    
    for yr in list_years:
        for subsec in list_subsectors:
            value = df.loc[(yr,country),subsec]
            if not math.isnan(value):
                dict_country={}

                dict_country["data_source"]={"name":"GCP"}
                dict_geo={}
                dict_geo["type"]="country"
                dict_geo["identifier"]={"id":df.loc[(2019,country),"ISO 3166-1 alpha-3"],"type":"alpha3"}
    
                dict_country["geo_component"]=dict_geo
                dict_country["date"]=str(yr)+"-01-01"
                dict_country["emission"]={"gas":"CO2","value":value,"unit":"MtC",
                                          "sector_name":"Territorial Emissions","subsector_name":subsec}
            
                #file.write(str(dict_country))
                json.dump(dict_country, file)
                file.write("\n")
